In [21]:
from bs4 import BeautifulSoup
import pandas as pd
import requests

url= 'https://scholar.google.com/scholar'

#set headers
headers = {
    'User-Agent': 'Mozilla/5.0 (Macintosh; Intel Mac OS X 10_10_1) \
    AppleWebKit/537.36 (KHTML, like Gecko) Chrome/39.0.2171.95 Safari/537.36' 
    #english
}

params = {
    #english
    'hl': 'en',

    'start': 0,
    'q': 'heat stress guidlines interventions mitigation strategies' 
}

In [22]:
##GET TITLE/LINK/AUTHORS OF N*10 ARTICLES

#loop through pages for n pages
n = 5

from tqdm.notebook import tqdm

#store results
results = pd.DataFrame(columns=['title', 'link', 'author'])

for i in tqdm(range(n)):
    params['start'] = i*10
    response = requests.get(url, headers=headers, params=params)
    soup = BeautifulSoup(response.text, 'html.parser')
    articles = soup.find_all('div', class_='gs_ri')

    #get titles
    titles = [article.find('h3', class_='gs_rt').text for article in articles]
    #get links
    links = [article.find('h3', class_='gs_rt').find('a')['href'] for article in articles]
    #get authors
    authors = [article.find('div', class_='gs_a').text for article in articles]
    
    #add to results
    results = results._append(pd.DataFrame({'title': titles, 'link': links, 'author': authors}))

  0%|          | 0/5 [00:00<?, ?it/s]

In [23]:
results.to_csv('article_info.csv')

In [24]:
## BASIC CODE TO SCRAPE ABSTRACT 

import requests
from bs4 import BeautifulSoup

def scrape_abstract(url):

    # YOUR CODE HERE
    response = requests.get(url)

    if response.status_code == 200:
        soup = BeautifulSoup(response.text, 'html.parser')

        abstract = soup.find('blockquote', class_='abstract').text.strip()
    
    return abstract

In [25]:
link = 'https://www.sciencedirect.com/science/article/pii/S0921800919302976'

response = requests.get(link, headers=headers)

print(response)

<Response [400]>


In [30]:
## CODE TO DOWNLOAD 1 LINK

from selenium import webdriver

from selenium.webdriver.common.keys import Keys
from selenium.webdriver.common.by import By

import time
import requests
from urllib.parse import urlparse

main_page_url = 'https://sci-hub.ru/'
url_to_search = 'https://www.sciencedirect.com/science/article/pii/S0921800919302976'
driver = webdriver.Chrome()

try:
    # Open page
    driver.get(main_page_url)
    time.sleep(2)  # Wait for the page to load (you can decrease this time if need be)

    #doi_input = driver.find_element()
    # Find search bar -> input URL
    search_bar = driver.find_element(by=By.ID, value='request')
    search_bar.send_keys(url_to_search)
    search_bar.send_keys(Keys.RETURN)  # To submit the search query
    time.sleep(1)  # Wait for a bit again

    # Find download link
    download_button = driver.find_element(by=By.XPATH, value="//button[@onclick]")
    onclick_attribute = download_button.get_attribute('onclick')

    # Extract URL from the onclick attribute
    # Assuming the format is location.href='URL'
    pdf_url = onclick_attribute.split("'")[1]
    
    download_link = 'https://' + urlparse(main_page_url).hostname + pdf_url
  
    # Download the PDF using requests
    response = requests.get(download_link)

    with open('pdf.pdf', 'wb') as file:
        file.write(response.content)

except Exception as e:
    # Handle the exception, print an error message, etc.
    print(f"An error occurred: {str(e)}")
finally:
    driver.quit()

In [31]:
#scraping function

from selenium import webdriver

from selenium.webdriver.common.keys import Keys
from selenium.webdriver.common.by import By

import time
import requests

def pdf_download(url_to_search, file_name):
    driver = webdriver.Chrome() #initialize web-finder
    main_page_url = 'https://sci-hub.ru/' #open sci-hub
    driver.get(main_page_url)
    time.sleep(1)
    # Find search bar -> input URL
    search_bar = driver.find_element(by=By.ID, value='request')
    search_bar.send_keys(url_to_search)
    search_bar.send_keys(Keys.RETURN)  # To submit the search query
    time.sleep(1)
    
    try:
        # Find download link
        download_button = driver.find_element(by=By.XPATH, value="//button[@onclick]")
        onclick_attribute = download_button.get_attribute('onclick')
    except:
        driver.quit()
        return url_to_search

    
    # Extract URL from the onclick attribute
    # Assuming the format is location.href='URL'
    pdf_url = onclick_attribute.split("'")[1]
    download_link = 'https://sci-hub.ru' + pdf_url
    # Download the PDF using requests
    response = requests.get(download_link)
    with open(f'pdfs/{file_name}.pdf', 'wb') as file:
        file.write(response.content)
    
    driver.quit()

    return None

In [32]:
##to call on all links !!! DO NOT RERUN, IT WILL SET LIST BACK TO NONE

import pandas as pd
import os

data = pd.read_csv('article_info.csv')
title = data['title']
links = data['link']
author = ['author'] 


def file_exists(file_path):
    return os.path.exists(file_path)

list_error_links = []

In [33]:
#function to call on all links

for index, row in data.iterrows():
    pdf_file_path = f'pdfs/{index}.pdf'
    if file_exists(pdf_file_path) is True:
        print("PDF already downloaded")
    else:
        if row['link'] in list_error_links:
            print('error already encountered')
        else:
            url_returned = pdf_download(row['link'], index)  #(links[index], index)
            if url_returned is not None:
                list_error_links.append(url_returned)
                print(f'ERROR FOR {url_returned}')

ERROR FOR https://www.jstage.jst.go.jp/article/indhealth/51/1/51_2012-0089/_article/-char/ja/
ERROR FOR https://journals.lww.com/joem/fulltext/2019/09000/Effect_of_Multivitamin_and_Multimineral.4.aspx
ERROR FOR https://www.aivc.org/resource/heat-waves-risks-and-responses
ERROR FOR https://www.researchgate.net/profile/Veerasamy-Sejian/publication/313548685_Behavioral_Responses_to_Livestock_Adaptation_to_Heat_Stress_Challenges/links/58c3d52fa6fdcce648e0a02d/Behavioral-Responses-to-Livestock-Adaptation-to-Heat-Stress-Challenges.pdf
ERROR FOR https://journals.uni-lj.si/aas/article/view/12783
ERROR FOR https://meridian.allenpress.com/jat/article-abstract/50/9/986/112280
ERROR FOR https://www.mdpi.com/1422-0067/18/9/1978
ERROR FOR https://journals.lww.com/co-clinicalnutrition/fulltext/2015/07000/Heat_shock_proteins_and_heat_therapy_for_type_2.10.aspx
ERROR FOR https://www.mdpi.com/1660-4601/18/9/4486
ERROR FOR https://jamanetwork.com/journals/jamainternalmedicine/article-abstract/413470
ERRO

NoSuchElementException: Message: no such element: Unable to locate element: {"method":"css selector","selector":"[id="request"]"}
  (Session info: chrome=134.0.6998.118); For documentation on this error, please visit: https://www.selenium.dev/documentation/webdriver/troubleshooting/errors#no-such-element-exception
Stacktrace:
	GetHandleVerifier [0x00007FF749554C25+3179557]
	(No symbol) [0x00007FF7491B88A0]
	(No symbol) [0x00007FF7490491CA]
	(No symbol) [0x00007FF74909FA67]
	(No symbol) [0x00007FF74909FC9C]
	(No symbol) [0x00007FF7490F3627]
	(No symbol) [0x00007FF7490C7C6F]
	(No symbol) [0x00007FF7490F02F3]
	(No symbol) [0x00007FF7490C7A03]
	(No symbol) [0x00007FF7490906D0]
	(No symbol) [0x00007FF749091983]
	GetHandleVerifier [0x00007FF7495B67CD+3579853]
	GetHandleVerifier [0x00007FF7495CD1D2+3672530]
	GetHandleVerifier [0x00007FF7495C2153+3627347]
	GetHandleVerifier [0x00007FF74932092A+868650]
	(No symbol) [0x00007FF7491C2FFF]
	(No symbol) [0x00007FF7491BF4A4]
	(No symbol) [0x00007FF74

In [34]:
%pip install pypdf

Note: you may need to restart the kernel to use updated packages.



[notice] A new release of pip is available: 23.2.1 -> 25.0.1
[notice] To update, run: C:\Users\Gur Levy\AppData\Local\Microsoft\WindowsApps\PythonSoftwareFoundation.Python.3.10_qbz5n2kfra8p0\python.exe -m pip install --upgrade pip


In [36]:
from pypdf import PdfReader
from os import listdir
from os.path import isfile, join

mypath = 'C:\\CAPSTONE\\scholar scraper\\pdfs'

pdf_folder_content = [f for f in listdir(mypath) if isfile(join(mypath, f))]

all_pdf_texts = []

for pdf in pdf_folder_content:
    print(f'Analyzing {pdf}...')
	
    reader = PdfReader(f'{mypath}/{pdf}')

    n_pages = len(reader.pages)
	
    print(f'{pdf} has {n_pages} pages!')

    all_text = []

    for i in range(0,n_pages):  
      page_text = reader.pages[i].extract_text()
      all_text.append(page_text)

    result_string = ' '.join(all_text)
    all_pdf_texts.append(result_string)

##SWITCH TO A DICTIONARY INSTEAD OF A LIST
# 

Analyzing 0.pdf...
0.pdf has 16 pages!


invalid pdf header: b'<!DOC'
EOF marker not found


Analyzing 10.pdf...


PdfStreamError: Stream has ended unexpectedly